In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import re
import pickle
from food_indexing import Indexer, IndexType, BasicInvertedIndex, InvertedIndex
from food_preprocessor import RegexTokenizer, Doc2QueryAugmenter

In [2]:
recipes = pd.read_csv('recipenlg.csv')
recipes.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [3]:
recipes.shape

(2231142, 7)

In [3]:
# recipes['newDirections'] = ""
# recipes['newIngredients'] = ""
recipes['website'] = ''

In [4]:
count = 0
import re

for idx, row in recipes.iterrows():
    if count <= 5000000:

        Nrow = row['directions']
        ingRow = row['ingredients']
        website = row['link']
        ner = row['NER']

        pattern = r'(?:(?:https?://)?(?:www\.)?)([\w.-]+)'

        match = re.search(pattern, website)

        if match:
            domain = match.group(1)
            recipes.at[idx, 'website'] = match.group(0)
        
        pattern = r'(?<=www\.)\w+\.\w+'
        domain = re.search(pattern, website)

        # if domain:
        #     #print(domain.group(0))
        #     recipes.at[idx, 'website'] = domain.group(0)
        
        newList = Nrow[2:-2].split('"')
        secList = ingRow[2:-2].split('"')
        nerList = ner[2:-2].split('"')
        for x in secList:
            if x == ', ':
                secList.remove(x)
        for x in newList:
            if x == ', ':
                newList.remove(x)
        for x in nerList:
            if x == ', ':
                nerList.remove(x)

        recipes.at[idx, 'directions'] = " ".join(newList)
        recipes.at[idx, 'ingredients'] = secList
        recipes.at[idx, 'NER'] = nerList
        #row['directions'] = " ".join(newList)
        # set newDirections to the new list
        # try:
        #     recipes.at[idx, 'newDirections'] = " ".join(newList)
        #     recipes.at[idx, 'newIngredients'] = secList
        # except:
        #     print(newList, secList)
        #print(secList)
        #print(" ".join(newList))
        # for row in row['directions']:
        #     print(row) 
    
    else:
    
        break
    count +=1

In [7]:
recipes.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,website
0,0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evapor...","In a heavy 2-quart saucepan, mix brown sugar, ...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[brown sugar, milk, vanilla, nuts, butter, bit...",www.cookbooks.com
1,1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chi...",Place chipped beef on bottom of baking dish. P...,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[beef, chicken breasts, cream of mushroom soup...",www.cookbooks.com
2,2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. c...","In a slow cooker, combine all ingredients. Cov...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[frozen corn, cream cheese, butter, garlic pow...",www.cookbooks.com
3,3,Chicken Funny,"[1 large whole chicken, 2 (10 1/2 oz.) cans ch...",Boil and debone chicken. Put bite size pieces ...,www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[chicken, chicken gravy, cream of mushroom sou...",www.cookbooks.com
4,4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker cru...",Combine first four ingredients and press in 13...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[peanut butter, graham cracker crumbs, butter,...",www.cookbooks.com


### things to consider -> how to get individual ingredients. Just use premade NER?

In [8]:
recipes.to_csv('cleanedRecipes.csv', index=False)


In [2]:
recipes = pd.read_csv('cleanedRecipes.csv')

In [3]:
recipes.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,website
0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva...","In a heavy 2-quart saucepan, mix brown sugar, ...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"['brown sugar', 'milk', 'vanilla', 'nuts', 'bu...",www.cookbooks.com
1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ...",Place chipped beef on bottom of baking dish. P...,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"['beef', 'chicken breasts', 'cream of mushroom...",www.cookbooks.com
2,2,Creamy Corn,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","In a slow cooker, combine all ingredients. Cov...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"['frozen corn', 'cream cheese', 'butter', 'gar...",www.cookbooks.com
3,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans...",Boil and debone chicken. Put bite size pieces ...,www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"['chicken', 'chicken gravy', 'cream of mushroo...",www.cookbooks.com
4,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ...",Combine first four ingredients and press in 13...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"['peanut butter', 'graham cracker crumbs', 'bu...",www.cookbooks.com


In [5]:
recipes.iloc[0]['ingredients']  

['1 c. firmly packed brown sugar',
 '1/2 c. evaporated milk',
 '1/2 tsp. vanilla',
 '1/2 c. broken nuts (pecans)',
 '2 Tbsp. butter or margarine',
 '3 1/2 c. bite size shredded rice biscuits']

In [6]:
recipes.rename(columns={'Unnamed: 0': 'recipeID'}, inplace=True)
recipes.head()

,recipeID,title,ingredients,directions,link,source,NER,website
0,0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evapor...","In a heavy 2-quart saucepan, mix brown sugar, ...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[brown sugar, milk, vanilla, nuts, butter, bit...",www.cookbooks.com
1,1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chi...",Place chipped beef on bottom of baking dish. P...,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[beef, chicken breasts, cream of mushroom soup...",www.cookbooks.com
2,2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. c...","In a slow cooker, combine all ingredients. Cov...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[frozen corn, cream cheese, butter, garlic pow...",www.cookbooks.com
3,3,Chicken Funny,"[1 large whole chicken, 2 (10 1/2 oz.) cans ch...",Boil and debone chicken. Put bite size pieces ...,www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[chicken, chicken gravy, cream of mushroom sou...",www.cookbooks.com
4,4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker cru...",Combine first four ingredients and press in 13...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[peanut butter, graham cracker crumbs, butter,...",www.cookbooks.com


In [13]:
# set recipe id type to int
recipes['recipeID'] = recipes['recipeID'].astype(int)
recipes['directions'] = recipes['directions'].astype(str)
recipes['website'] = recipes['website'].astype(str)
recipes['source'] = recipes['source'].astype(str)


In [14]:
recipes.dtypes

recipeID        int64
title          object
ingredients    object
directions     object
link           object
source         object
NER            object
website        object
dtype: object

In [7]:
with open("cleanedRecipe", "wb") as fp:   #Pickling
    pickle.dump(recipes, fp)

In [23]:
with open("cleanedRecipe", "rb") as fp:   # Unpickling
    recipes = pickle.load(fp)

In [10]:
recipes[0]

KeyError: 0

In [19]:
recipes

,Unnamed: 0,title,ingredients,directions,link,source,NER,website
0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva...","In a heavy 2-quart saucepan, mix brown sugar, ...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"['brown sugar', 'milk', 'vanilla', 'nuts', 'bu...",www.cookbooks.com
1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ...",Place chipped beef on bottom of baking dish. P...,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"['beef', 'chicken breasts', 'cream of mushroom...",www.cookbooks.com
2,2,Creamy Corn,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","In a slow cooker, combine all ingredients. Cov...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"['frozen corn', 'cream cheese', 'butter', 'gar...",www.cookbooks.com
3,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans...",Boil and debone chicken. Put bite size pieces ...,www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"['chicken', 'chicken gravy', 'cream of mushroo...",www.cookbooks.com
4,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ...",Combine first four ingredients and press in 13...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"['peanut butter', 'graham cracker crumbs', 'bu...",www.cookbooks.com
...,...,...,...,...,...,...,...,...
2231137,2231137,Sunny's Fake Crepes,['1/2 cup chocolate hazelnut spread (recommend...,Spread hazelnut spread on 1 side of each torti...,www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"['chocolate hazelnut spread', 'tortillas', 'bu...",www.foodnetwork.com
2231138,2231138,Devil Eggs,"['1 dozen eggs', '1 paprika', '1 salt and pepp...",Boil eggs on medium for 30mins. Then cool eggs...,cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"['eggs', 'paprika', 'salt', 'choice', 'miracle...",cookpad.com
2231139,2231139,Extremely Easy and Quick - Namul Daikon Salad,"['150 grams Daikon radish', '1 tbsp Sesame oil...",Julienne the daikon and squeeze out the excess...,cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"['radish', 'Sesame oil', 'White sesame seeds',...",cookpad.com
2231140,2231140,Pan-Roasted Pork Chops With Apple Fritters,"['1 cup apple cider', '6 tablespoons sugar', '...","In a large bowl, mix the apple cider with 4 cu...",cooking.nytimes.com/recipes/1015164,Recipes1M,"['apple cider', 'sugar', 'kosher salt', 'bay l...",cooking.nytimes.com


In [20]:
recipes.to_json('cleanedRecipes.json')

In [24]:
#check dtypes of recipe colums
recipes.dtypes


Unnamed: 0      int64
title          object
ingredients    object
directions     object
link           object
source         object
NER            object
website        object
dtype: object

In [25]:
doclist = []

In [21]:
doclist = []
import json
count = 0
with open('cleanedRecipes.json') as f:
    # for line in (f):
    doclist.append(json.load(f))

  

In [23]:
doclist[0]

{'Unnamed: 0': {'0': 0,
  '1': 1,
  '2': 2,
  '3': 3,
  '4': 4,
  '5': 5,
  '6': 6,
  '7': 7,
  '8': 8,
  '9': 9,
  '10': 10,
  '11': 11,
  '12': 12,
  '13': 13,
  '14': 14,
  '15': 15,
  '16': 16,
  '17': 17,
  '18': 18,
  '19': 19,
  '20': 20,
  '21': 21,
  '22': 22,
  '23': 23,
  '24': 24,
  '25': 25,
  '26': 26,
  '27': 27,
  '28': 28,
  '29': 29,
  '30': 30,
  '31': 31,
  '32': 32,
  '33': 33,
  '34': 34,
  '35': 35,
  '36': 36,
  '37': 37,
  '38': 38,
  '39': 39,
  '40': 40,
  '41': 41,
  '42': 42,
  '43': 43,
  '44': 44,
  '45': 45,
  '46': 46,
  '47': 47,
  '48': 48,
  '49': 49,
  '50': 50,
  '51': 51,
  '52': 52,
  '53': 53,
  '54': 54,
  '55': 55,
  '56': 56,
  '57': 57,
  '58': 58,
  '59': 59,
  '60': 60,
  '61': 61,
  '62': 62,
  '63': 63,
  '64': 64,
  '65': 65,
  '66': 66,
  '67': 67,
  '68': 68,
  '69': 69,
  '70': 70,
  '71': 71,
  '72': 72,
  '73': 73,
  '74': 74,
  '75': 75,
  '76': 76,
  '77': 77,
  '78': 78,
  '79': 79,
  '80': 80,
  '81': 81,
  '82': 82,
  '83': 83

In [6]:
pd.read_json('cleanedRecipes.json')

,Unnamed: 0,title,ingredients,directions,link,source,NER,website
0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva...","In a heavy 2-quart saucepan, mix brown sugar, ...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"['brown sugar', 'milk', 'vanilla', 'nuts', 'bu...",www.cookbooks.com
1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ...",Place chipped beef on bottom of baking dish. P...,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"['beef', 'chicken breasts', 'cream of mushroom...",www.cookbooks.com
2,2,Creamy Corn,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","In a slow cooker, combine all ingredients. Cov...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"['frozen corn', 'cream cheese', 'butter', 'gar...",www.cookbooks.com
3,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans...",Boil and debone chicken. Put bite size pieces ...,www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"['chicken', 'chicken gravy', 'cream of mushroo...",www.cookbooks.com
4,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ...",Combine first four ingredients and press in 13...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"['peanut butter', 'graham cracker crumbs', 'bu...",www.cookbooks.com
...,...,...,...,...,...,...,...,...
2231137,2231137,Sunny's Fake Crepes,['1/2 cup chocolate hazelnut spread (recommend...,Spread hazelnut spread on 1 side of each torti...,www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"['chocolate hazelnut spread', 'tortillas', 'bu...",www.foodnetwork.com
2231138,2231138,Devil Eggs,"['1 dozen eggs', '1 paprika', '1 salt and pepp...",Boil eggs on medium for 30mins. Then cool eggs...,cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"['eggs', 'paprika', 'salt', 'choice', 'miracle...",cookpad.com
2231139,2231139,Extremely Easy and Quick - Namul Daikon Salad,"['150 grams Daikon radish', '1 tbsp Sesame oil...",Julienne the daikon and squeeze out the excess...,cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"['radish', 'Sesame oil', 'White sesame seeds',...",cookpad.com
2231140,2231140,Pan-Roasted Pork Chops With Apple Fritters,"['1 cup apple cider', '6 tablespoons sugar', '...","In a large bowl, mix the apple cider with 4 cu...",cooking.nytimes.com/recipes/1015164,Recipes1M,"['apple cider', 'sugar', 'kosher salt', 'bay l...",cooking.nytimes.com


In [4]:
recipes.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,website
0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva...","In a heavy 2-quart saucepan, mix brown sugar, ...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"['brown sugar', 'milk', 'vanilla', 'nuts', 'bu...",www.cookbooks.com
1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ...",Place chipped beef on bottom of baking dish. P...,www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"['beef', 'chicken breasts', 'cream of mushroom...",www.cookbooks.com
2,2,Creamy Corn,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","In a slow cooker, combine all ingredients. Cov...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"['frozen corn', 'cream cheese', 'butter', 'gar...",www.cookbooks.com
3,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans...",Boil and debone chicken. Put bite size pieces ...,www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"['chicken', 'chicken gravy', 'cream of mushroo...",www.cookbooks.com
4,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ...",Combine first four ingredients and press in 13...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"['peanut butter', 'graham cracker crumbs', 'bu...",www.cookbooks.com


In [8]:
output_file = 'cleanedRecipes.jsonl'

# Define a function to convert a DataFrame row to a JSON record
def row_to_jsonl(row):
    json_record = row.to_dict()
    json_record['NER'] = json.dumps(row['NER'])
    json_record['ingredients'] = json.dumps(row['ingredients'])
    return json.dumps(json_record)

# Write the DataFrame to a JSONL file
with open(output_file, 'w') as file:
    for index, row in recipes.iterrows():
        json_record = row_to_jsonl(row)
        file.write(json_record + '\n')

In [2]:
doclist = []
with open('cleanedRecipes.jsonl') as f:
    for line in (f):
        json_record = json.loads(line)
        json_record['ingredients'] = json.loads(json_record['ingredients'])
        json_record['NER'] = json.loads(json_record['NER'])
        doclist.append(json_record)

In [5]:
doclist[0]

{'recipeID': 0,
 'title': 'No-Bake Nut Cookies',
 'ingredients': ['1 c. firmly packed brown sugar',
  '1/2 c. evaporated milk',
  '1/2 tsp. vanilla',
  '1/2 c. broken nuts (pecans)',
  '2 Tbsp. butter or margarine',
  '3 1/2 c. bite size shredded rice biscuits'],
 'directions': 'In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.',
 'link': 'www.cookbooks.com/Recipe-Details.aspx?id=44874',
 'source': 'Gathered',
 'NER': ['brown sugar',
  'milk',
  'vanilla',
  'nuts',
  'butter',
  'bite size shredded rice biscuits'],
 'website': 'www.cookbooks.com'}

In [24]:
recipes.iloc[0]['ingredients']  

"['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 Tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits']"

In [ ]:
with open('cleanedRecipes.csv', 'r') as f:
    data = f.read()
    

In [3]:
recipes['website'].value_counts()

website
www.cookbooks.com          896341
www.food.com               499616
www.epicurious.com         129444
tastykitchen.com            78768
www.myrecipes.com           64895
www.allrecipes.com          61398
cookpad.com                 61020
cookeatshare.com            59307
www.yummly.com              51963
www.tasteofhome.com         51594
www.foodnetwork.com         49443
food52.com                  48501
www.kraftrecipes.com        42010
recipeland.com              24418
recipes-plus.com            20524
cooking.nytimes.com         16367
www.foodandwine.com         15436
www.seriouseats.com         12632
www.foodgeeks.com            8963
www.cookstr.com              8797
online-cookbook.com          5691
www.chowhound.com            5671
www.vegetariantimes.com      4578
www.delish.com               3880
allrecipes.com               3204
www.landolakes.com           2492
www.foodrepublic.com         2259
www.lovefood.com             1930
Name: count, dtype: int64

In [ ]:
recipes[recipes['source'] == 'Recipes1M']

,Unnamed: 0,title,ingredients,directions,link,source,NER,newDirections,newIngredients,website
1643098,1643098,Worlds Best Mac and Cheese,"[""6 ounces penne"", ""2 cups Beechers Flagship C...","[""Preheat the oven to 350 F. Butter or oil an ...",www.epicurious.com/recipes/food/views/-world-s...,Recipes1M,"[""penne"", ""Beechers Flagship Cheese Sauce"", ""C...",0,0,epicurious.com
1643099,1643099,Dilly Macaroni Salad Recipe,"[""1 c. elbow macaroni"", ""1 c. cubed American c...","[""Cook macaroni according to package direction...",cookeatshare.com/recipes/dilly-macaroni-salad-...,Recipes1M,"[""elbow macaroni"", ""American cheese"", ""celery""...",0,0,
1643100,1643100,Gazpacho,"[""8 tomatoes, quartered"", ""Kosher salt"", ""1 re...","[""Add the tomatoes to a food processor with a ...",www.foodnetwork.com/recipes/gazpacho1.html,Recipes1M,"[""tomatoes"", ""Kosher salt"", ""red onion"", ""gree...",0,0,foodnetwork.com
1643101,1643101,Kombu Tea Grilled Chicken Thigh,"[""2 Chicken thighs"", ""2 tsp Kombu tea"", ""1 Whi...","[""Pierce the skin of the chicken with a fork o...",cookpad.com/us/recipes/150100-kombu-tea-grille...,Recipes1M,"[""Chicken"", ""White pepper""]",0,0,
1643102,1643102,Zucchini Nut Bread,"[""2 cups flour"", ""1 tablespoon cinnamon"", ""2 t...","[""Sift dry ingredients."", ""beat eggs untill fr...",www.food.com/recipe/zucchini-nut-bread-329682,Recipes1M,"[""flour"", ""cinnamon"", ""baking soda"", ""salt"", ""...",0,0,food.com
...,...,...,...,...,...,...,...,...,...,...
2231137,2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu...",0,0,foodnetwork.com
2231138,2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle...",0,0,
2231139,2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",...",0,0,
2231140,2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l...",0,0,


In [20]:
newList = Nrow[2:-2].split('"')

In [35]:
ingRow[2:-2].split('"')

['1 c. firmly packed brown sugar',
 ', ',
 '1/2 c. evaporated milk',
 ', ',
 '1/2 tsp. vanilla',
 ', ',
 '1/2 c. broken nuts (pecans)',
 ', ',
 '2 Tbsp. butter or margarine',
 ', ',
 '3 1/2 c. bite size shredded rice biscuits']

In [22]:
for x in newList:
    if x == ', ':
        newList.remove(x)

In [25]:
" ".join(newList)

'In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.'

## slay time

In [1]:
from importlib import reload
import food_indexing
import food_preprocessor
reload(food_indexing)
reload(food_preprocessor)
from food_indexing import Indexer, IndexType, BasicInvertedIndex, InvertedIndex
from food_preprocessor import RegexTokenizer, Doc2QueryAugmenter

/Users/bellakarduck/Downloads/si 650/si650/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = 'cleanedRecipes.jsonl'
stopwords = {'and', 'the', 'or', 'is', 'for'}
text_key = 'directions'
doc_augment_dict = {}
document_preprocessor = RegexTokenizer('/w+')
minimum_word_frequency = 5

In [6]:
setOfStopwords = {'and', 'the', 'or', 'could', 'if'}
preprocessor = RegexTokenizer('\w+', lowercase=True, multiword_expressions=None)
text_index = Indexer.create_index(IndexType.InvertedIndex, dataset_path='cleanedRecipes.jsonl', document_preprocessor=preprocessor, stopwords=setOfStopwords, minimum_word_frequency=4, text_key='directions', max_docs=100000)
print(text_index.get_statistics())

2231142it [00:12, 176885.92it/s]
  4%|▍         | 100000/2231142 [00:01<00:27, 76372.02it/s]


Counter({'and': 238299, 'in': 136194, 'to': 89532, 'add': 83033, 'with': 71042, 'until': 67979, 'minutes': 66930, 'a': 62151, 'mix': 60031, 'for': 59387, '1': 55644, 'the': 52341, 'u00b0': 48842, 'bake': 48595, 'of': 45774, 'at': 45635, 'into': 42923, '2': 40558, 'on': 37147, 'sugar': 34854, 'ingredients': 34853, 'or': 34696, 'over': 34491, 'pour': 31815, 'mixture': 30778, 'pan': 30111, 'stir': 30003, '350': 29209, 'well': 27023, 'cheese': 26823, 'cook': 26521, 'butter': 24808, 'water': 24534, 'heat': 24102, 'salt': 23937, 'top': 23726, 'inch': 23015, 'flour': 22643, 'cream': 22570, 'combine': 22026, 'together': 21752, 'all': 20270, 'place': 19657, 'brown': 18923, 'oven': 17962, 'milk': 17802, 'cool': 17620, 'bowl': 17481, 'baking': 17163, 'beat': 16700, 'x': 16605, '4': 16310, '3': 16097, 'cover': 15656, 'eggs': 15226, 'sprinkle': 15175, 'dish': 15131, 'onion': 14967, 'put': 14847, 'chicken': 14797, 'pepper': 14496, 'large': 14180, 'serve': 13982, '10': 13952, '9': 13820, 'about': 137

  4%|▍         | 100000/2231142 [00:02<00:56, 37584.19it/s]


defaultdict(<class 'collections.Counter'>, {'index_type': 'BasicInvertedIndex', 'unique_token_count': 4979, 'total_token_count': 4352379, 'number_of_documents': 100000, 'mean_document_length': 43.52379})


In [8]:
text_index.get_statistics()

defaultdict(collections.Counter,
            {'index_type': 'BasicInvertedIndex',
             'unique_token_count': 4979,
             'total_token_count': 4352379,
             'number_of_documents': 100000,
             'mean_document_length': 43.52379})

In [9]:
text_index.index['cook']

[(2, 1),
 (5, 4),
 (23, 3),
 (26, 1),
 (40, 1),
 (47, 1),
 (56, 1),
 (70, 1),
 (76, 1),
 (78, 1),
 (80, 1),
 (88, 1),
 (95, 3),
 (96, 1),
 (100, 1),
 (101, 1),
 (102, 1),
 (106, 1),
 (114, 1),
 (115, 2),
 (116, 2),
 (119, 1),
 (121, 2),
 (129, 2),
 (136, 1),
 (138, 1),
 (140, 1),
 (141, 1),
 (143, 1),
 (144, 2),
 (147, 2),
 (171, 1),
 (174, 1),
 (179, 1),
 (181, 1),
 (182, 2),
 (183, 1),
 (193, 1),
 (198, 2),
 (202, 1),
 (205, 1),
 (210, 1),
 (224, 1),
 (241, 1),
 (245, 1),
 (247, 1),
 (253, 2),
 (262, 1),
 (265, 1),
 (267, 2),
 (270, 2),
 (276, 1),
 (287, 2),
 (301, 3),
 (303, 1),
 (304, 1),
 (306, 2),
 (314, 1),
 (318, 1),
 (319, 2),
 (328, 2),
 (331, 2),
 (333, 1),
 (336, 1),
 (344, 1),
 (348, 1),
 (353, 3),
 (354, 1),
 (372, 1),
 (376, 1),
 (384, 1),
 (390, 1),
 (393, 1),
 (401, 2),
 (404, 1),
 (405, 1),
 (417, 1),
 (418, 1),
 (423, 1),
 (427, 1),
 (428, 1),
 (433, 1),
 (442, 1),
 (464, 1),
 (465, 2),
 (470, 2),
 (479, 2),
 (481, 2),
 (484, 1),
 (489, 1),
 (494, 1),
 (498, 1),
 (49

In [10]:
text_index.get_postings('chicken')[:10]

[(1, 2),
 (3, 3),
 (12, 1),
 (31, 1),
 (40, 1),
 (47, 2),
 (50, 1),
 (63, 1),
 (76, 1),
 (78, 1)]

In [13]:
import food_ranker


In [18]:
ranker = food_ranker.Ranker(text_index, preprocessor, stopwords, BM25)
topq = ranker.query('I want a casserole with chicken')[:10]
print(topq)

[(82978, 15.299228342917768), (23265, 15.174591822683487), (77298, 13.507116836517541), (74525, 13.206984060735902), (96767, 12.46549795535644), (58255, 12.218907007333385), (76459, 11.873095483101098), (72752, 11.713511046163061), (64966, 11.566081509139249), (58127, 11.452188958115272)]


In [19]:
for q in topq:
    with open('cleanedRecipes.jsonl') as f:
        for line in (f):
            json_record = json.loads(line)
            if q[0] == json_record['recipeID']:
                print(json_record['directions'])
               
                print(json_record['title'])

    print(q)

The Lord is my Shepherd I shall not want.
Texas Trash
(82978, 15.299228342917768)
Boil chicken until done, then debone. Add can of cream of chicken soup; bring to a boil. Then add noodles; add chicken. I add can of peas for color. You don't have to if you don't want to.
Chicken And Noodles
(23265, 15.174591822683487)
Put hominy (I like yellow), butter, salt, pepper and grated sharp cheese in layers in heavy casserole. Bake until cheese has melted and is bubbly. You can layer as many times as you want. (Excellent for a cold winter night meal.)
Hominy Casserole
(77298, 13.507116836517541)
Saute onion, celery and carrots in butter. Simmer. Add other ingredients. Cut chicken into approximately 1-inch cubes and brown in butter or peanut oil. After all is cooked, may want to thicken with cornstarch. I use my wok for this dish.
Chow Mein
(74525, 13.206984060735902)
Skin chicken breasts or legs. Boil until cooked. Chop in cubes. In chicken broth, cook celery and broccoli, just a few minutes. Y

KeyboardInterrupt: 

In [27]:
from food_ranker import *
ranker = Ranker(food_index, document_preprocessor, stopwords, TF_IDF)
topq = ranker.query('can i eat some chicken')[:10]

In [28]:
topq

[]

In [19]:
for q in topq:
    print(q)